In [1]:
import numpy as np
import pandas as pd
from scipy.fft import fft, ifft
from scipy.interpolate import interp1d
from scipy.stats import entropy, iqr, kurtosis, mode, skew
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
import tensorflow as tf
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, VotingClassifier
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import (Activation, BatchNormalization, Bidirectional,
                                     Conv1D, Conv2D, Dense, Dropout,
                                     Flatten, GlobalAveragePooling1D, LSTM,
                                     MaxPool1D, MaxPooling2D, Reshape,
                                     TimeDistributed)
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import os
import pickle
from pandas import read_csv, unique
from scipy.signal import argrelextrema, butter, find_peaks, lfilter, lfilter_zi
from sklearn2pmml import sklearn2pmml
from sklearn2pmml.pipeline import PMMLPipeline


2023-11-02 00:14:44.521265: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-02 00:14:44.732651: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-11-02 00:14:44.732687: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-11-02 00:14:45.514812: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [2]:
X_respeck = pickle.load(open("X_respeck", "rb"))
y_respeck = pickle.load(open("y_respeck", "rb"))

In [3]:
X_train, X_temp, y_train, y_temp = train_test_split(X_respeck, y_respeck, test_size=0.2, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

In [4]:
enc = OneHotEncoder(handle_unknown='ignore', sparse=False)

enc = enc.fit(np.array(y_train))
y_train = enc.transform(y_train)
y_val = enc.transform(y_val)
y_test = enc.transform(y_test)


In [5]:
# create individual models
cnn_lstm = Sequential()
cnn_lstm.add(LSTM(32, return_sequences=True, input_shape=X_train[0].shape, activation='relu'))
cnn_lstm.add(LSTM(32,return_sequences=True, activation='relu'))
cnn_lstm.add(Conv1D(filters=64,kernel_size=2, activation='relu', strides=2))
cnn_lstm.add(MaxPool1D(pool_size=4, padding='same'))
cnn_lstm.add(Conv1D(filters=192, kernel_size=2, activation='relu', strides=1))
cnn_lstm.add(GlobalAveragePooling1D())
cnn_lstm.add(BatchNormalization(epsilon=1e-06))
cnn_lstm.add(Dense(12))
cnn_lstm.add(Activation('softmax'))
cnn_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
cnn_lstm.fit(X_train, y_train, epochs=10, batch_size=32, verbose=1)

2023-11-02 00:14:53.323863: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-11-02 00:14:53.323887: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-11-02 00:14:53.323904: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dan-HP-ENVY-x360-Convertible-13-ay0xxx): /proc/driver/nvidia/version does not exist
2023-11-02 00:14:53.324286: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/10


2023-11-02 00:14:53.700713: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 91736000 exceeds 10% of free system memory.


 351/1434 [======>.......................] - ETA: 35s - loss: 0.5919 - accuracy: 0.7788

KeyboardInterrupt: 

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(cnn_lstm)
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
converter._experimental_lower_tensor_list_ops = False
tflite_model = converter.convert()

# Save the model.
with open('C:/Users/Public/cnn_lstm.tflite', 'wb') as f:
  f.write(tflite_model)

In [53]:
rf_model = RandomForestClassifier(random_state=42)

X_train_rf = X_train.reshape(X_train.shape[0], -1)
# Train the model
rf_model.fit(X_train_rf, np.argmax(y_train, axis=1))

RandomForestClassifier(random_state=42)

In [82]:
import skl2onnx
import onnxruntime as rt
from skl2onnx.common.data_types import DoubleTensorType
from skl2onnx import convert_sklearn

initial_type = [('float_input', DoubleTensorType([None, X_train_rf.shape[1]]))]
onx = convert_sklearn(rf_model, initial_types=initial_type)
with open("rf_model.onnx", "wb") as f:
    f.write(onx.SerializeToString())

In [83]:
import onnx
onnx_model = onnx.load("rf_model.onnx")
onnx.checker.check_model(onnx_model)
sess = rt.InferenceSession("rf_model.onnx")
input_name = sess.get_inputs()[0].name
label_name = sess.get_outputs()[0].name
pred_onx = sess.run([label_name], {input_name: X_train_rf[0].reshape(1,-1)})[0]

BackendIsNotSupposedToImplementIt: in user code:

    File "C:\Users\danin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\onnx_tf\backend_tf_module.py", line 99, in __call__  *
        output_ops = self.backend._onnx_node_to_tensorflow_op(onnx_node,
    File "C:\Users\danin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\onnx_tf\backend.py", line 349, in _onnx_node_to_tensorflow_op  *
        raise BackendIsNotSupposedToImplementIt("{} is not implemented.".format(

    BackendIsNotSupposedToImplementIt: TreeEnsembleClassifier is not implemented.


In [ ]:
# how we combine:
def full_stack_model(X):
        # show confusion matrix
        X_rf = X.reshape(X.shape[0], -1)
        y_pred_cnn_lstm = 1.3*cnn_lstm.predict(X) # weight cnn_lstm more heavily - this was found by grid search
        y_pred_rf = rf_model_pipeline.predict(X_rf)
        y_pred_ensemble = np.mean(np.array([y_pred_cnn_lstm, y_pred_rf]), axis=0)
        y_pred_ensemble = np.argmax(y_pred_ensemble, axis=1)
        # correct for known issues with sit stand detection
        for i, p in enumerate(y_pred_ensemble):
            if p == 10 or p == 11:
                y_pred_ensemble[i] = np.argmax(y_pred_rf[i])
            if y_pred_ensemble[i] == 0:
                y_pred_ensemble[i] = np.argmax(y_pred_cnn_lstm[i])
        return y_pred_ensemble 

